In [2]:
import pandas as pd
import tldextract
import numpy as np

# ---------------------------------------------
# 1) Cargar dataset base v21
# ---------------------------------------------
PATH = "../data/clean/dataset_base_v21.csv"
df = pd.read_csv(PATH)

df = df.copy()

# ---------------------------------------------
# 2) Cargar whitelist española (registered_domain)
# ✏️ Ajusta la ruta si tu whitelist está en otra carpeta
# ---------------------------------------------
WHITELIST_PATH = "../docs/whitelist.csv"  # cámbiala si hace falta
wl = pd.read_csv(WHITELIST_PATH, header=None)[0].str.lower().tolist()
wl = set(wl)

# ---------------------------------------------
# 3) Extraer registered_domain correcto
# ---------------------------------------------
def get_registered_domain(url):
    ext = tldextract.extract(url)
    if ext.domain and ext.suffix:
        return f"{ext.domain.lower()}.{ext.suffix.lower()}"
    return ""

df["registered_domain"] = df["url"].apply(get_registered_domain)

# ---------------------------------------------
# 4) Calcular domain_whitelist_score REAL
# ---------------------------------------------
df["domain_whitelist_calc"] = df["registered_domain"].isin(wl).astype(int)

# ---------------------------------------------
# 5) Métricas por clase
# ---------------------------------------------
print("Distribución por clase:")
whitelist_stats = df.groupby("label")["domain_whitelist_calc"].mean().to_frame("whitelist_rate")
display(whitelist_stats)

# ---------------------------------------------
# 6) Ejemplos whitelist = 1 (deben ser dominios .es legítimos)
# ---------------------------------------------
print("\nEjemplos donde la whitelist se activa:")
display(df[df["domain_whitelist_calc"] == 1].head(10)[["url", "registered_domain", "label"]])

# ---------------------------------------------
# 7) Posibles falsos positivos: whitelist = 1 en phishing
# ---------------------------------------------
print("\nPosibles FALSOS POSITIVOS (debe estar vacío):")
display(df[(df["domain_whitelist_calc"] == 1) & (df["label"] == 1)].head(10)[["url", "registered_domain"]])

# ---------------------------------------------
# 8) Similitudes peligrosas NO whitelist (es correcto que sean 0)
# ---------------------------------------------
mask = df["registered_domain"].str.contains("bbva|santander|caixa|correos|cetelem|wizink", na=False)
print("\nDominios PARECIDOS pero NO whitelist (deben ser 0):")
display(df[(df["domain_whitelist_calc"] == 0) & mask].head(10)[["url", "registered_domain", "label"]])


Distribución por clase:


,whitelist_rate
label,
0,0.713115
1,0.024194



Ejemplos donde la whitelist se activa:


,url,registered_domain,label
0,https://www.caixabank.es/particular/banca-digi...,caixabank.es,0
1,https://www.ibercaja.es/particulares/,ibercaja.es,0
7,https://www.ibercaja.es/empresas/,ibercaja.es,0
9,https://portal.kutxabank.es/cs/Satellite/kb/es...,kutxabank.es,0
10,https://portal.kutxabank.es/cs/Satellite/kb/es...,kutxabank.es,0
11,https://www.unicajabanco.es/es/particulares/hi...,unicajabanco.es,0
12,https://www.unicajabanco.es/es/empresas-y-auto...,unicajabanco.es,0
13,https://www.unicajabanco.es/es/banca-digital,unicajabanco.es,0
15,https://portal.kutxabank.es/cs/Satellite/kb/es...,kutxabank.es,0
16,https://www.myinvestor.es/,myinvestor.es,0



Posibles FALSOS POSITIVOS (debe estar vacío):


,url,registered_domain
371,https://serviciodecorreo.es/,serviciodecorreo.es
372,https://sites.google.com/view/habilitaciondeac...,google.com
373,https://sites.google.com/view/serviciosexclusi...,google.com
401,https://sites.google.com/view/confirmaciondeda...,google.com
413,https://sites.google.com/view/ruraless/p%C3%A1...,google.com
461,http://unicismadrid.es/wp-content/com/index/ch...,unicismadrid.es



Dominios PARECIDOS pero NO whitelist (deben ser 0):


,url,registered_domain,label
2,https://www.wizink.es/tarjetas,wizink.es,0
3,https://www.wizink.es/,wizink.es,0
4,https://www.cetelem.es/credito-y-prestamos/,cetelem.es,0
5,https://www.cetelem.es/,cetelem.es,0
41,https://www.bbva.com/es/,bbva.com,0
255,https://sncrly-bbva.xyz/bbva/area_cliente/smsc...,sncrly-bbva.xyz,1
274,http://envios-sinotrans-correos.com/seguimient...,envios-sinotrans-correos.com,1
416,https://aviso-caixa.digital/inicio/enter,aviso-caixa.digital,1
418,https://santanderib.com,santanderib.com,1
480,https://correos-paqueteria.com/asset.php,correos-paqueteria.com,1
